# Generación de Datos Sintéticos - Proyecto de Grado
## Framework de RL para Optimización de Datos Sintéticos y Modelos ML

**Título:** Sintetizando datos tabulares: Un Framework de Aprendizaje por Refuerzo para el Benchmark de Datos Sintéticos y su Impacto en problemas de Clasificación

**Autor:** Carlos Andres Cortez Ballen

### Objetivos de este Notebook:
- Implementar y comparar generadores de datos sintéticos (GANs vs métodos tradicionales)
- Evaluar calidad de datos sintéticos usando métricas definidas en el proyecto
- Generar datos sintéticos para el segmento D (baja transaccionalidad)
- Preparar datos para entrenamiento de modelos ML
- Establecer baseline de calidad para datos sintéticos

### Metodología:
1. **Generadores GAN**: CTGAN, TVAE
2. **Generadores Tradicionales**: Gaussian Copula, SMOTE, ADASYN
3. **Métricas de Calidad**: KS Complement, Chi-Squared, KL Divergence, Cosine Similarity, Jensen-Shannon Entropy
4. **Validación**: 15-20% de datos para validación de sintetizadores

### Contenido:
1. Configuración y carga de datos
2. Implementación de generadores sintéticos
3. Entrenamiento de generadores
4. Generación de datos sintéticos
5. Evaluación de calidad
6. Comparación de generadores
7. Preparación para modelado


In [1]:
# Importación de librerías y configuración
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import yaml
import logging
from tqdm import tqdm

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Cargar configuración
with open('../configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✅ Librerías importadas correctamente")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")
print(f"⚙️ Configuración cargada: {len(config)} secciones")


✅ Librerías importadas correctamente
📊 Pandas version: 2.2.3
🔢 NumPy version: 1.26.4
📈 Matplotlib version: 3.10.1
🎨 Seaborn version: 0.13.2
⚙️ Configuración cargada: 8 secciones


In [2]:
# Importar módulos del proyecto
from src.data.data_loader import DataLoader
from src.data.data_preprocessor import DataPreprocessor
from src.data.data_splitter import DataSplitter
from src.synthetic_generators.generator_factory import GeneratorFactory
from src.synthetic_generators.quality_evaluator import SyntheticQualityEvaluator

# Inicializar componentes
data_loader = DataLoader(config)
data_preprocessor = DataPreprocessor(config)
data_splitter = DataSplitter(config)
generator_factory = GeneratorFactory(config)
quality_evaluator = SyntheticQualityEvaluator(config)

print("✅ Módulos del proyecto importados correctamente")
print("📁 DataLoader inicializado")
print("🔧 DataPreprocessor inicializado")
print("✂️ DataSplitter inicializado")
print("🏭 GeneratorFactory inicializado")
print("📊 QualityEvaluator inicializado")


2025-10-17 13:34:21,932 - INFO - DataSplitter inicializado:
2025-10-17 13:34:21,942 - INFO -   Train: 0.6, Validation: 0.2, Test: 0.2
2025-10-17 13:34:21,942 - INFO -   Synthetic validation: 0.15
2025-10-17 13:34:21,945 - INFO -   CV folds: 5
2025-10-17 13:34:21,947 - INFO - GeneratorFactory inicializado
2025-10-17 13:34:21,949 - INFO -   GAN models: ['CTGAN', 'TVAE', 'WGAN', 'DCGAN']
2025-10-17 13:34:21,949 - INFO -   Traditional models: ['GaussianCopula', 'SMOTE', 'ADASYN', 'BorderlineSMOTE']
2025-10-17 13:34:21,951 - INFO - Evaluador de calidad inicializado con 5 métricas


✅ Módulos del proyecto importados correctamente
📁 DataLoader inicializado
🔧 DataPreprocessor inicializado
✂️ DataSplitter inicializado
🏭 GeneratorFactory inicializado
📊 QualityEvaluator inicializado


In [3]:
# Cargar datos del segmento D (usando German Credit como proxy)
print("🔄 Cargando datos del segmento D...")

try:
    # Cargar German Credit Data como proxy del segmento D
    german_features, german_targets = data_loader.load_uci_dataset(144, "german_credit")
    segment_d_data = pd.concat([german_features, german_targets], axis=1)
    
    print(f"✅ Datos del segmento D cargados: {segment_d_data.shape}")
    print(f"   Features: {german_features.shape[1]}")
    print(f"   Target distribution: {german_targets.iloc[:, 0].value_counts().to_dict()}")
    
    # Mostrar información básica
    print(f"\n📊 Información del dataset:")
    print(f"   Shape: {segment_d_data.shape}")
    print(f"   Memory usage: {segment_d_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   Missing values: {segment_d_data.isnull().sum().sum()}")
    print(f"   Duplicates: {segment_d_data.duplicated().sum()}")
    
    # Mostrar primeras filas
    print(f"\n📖 Primeras 5 filas:")
    display(segment_d_data.head())
    
except Exception as e:
    print(f"❌ Error cargando datos: {e}")
    segment_d_data = None


2025-10-17 13:34:31,566 - INFO - Cargando dataset UCI ID: 144


🔄 Cargando datos del segmento D...


2025-10-17 13:34:33,448 - INFO - Dataset cargado: 1000 filas, 20 features
2025-10-17 13:34:33,450 - INFO - Target shape: (1000, 1)


✅ Datos del segmento D cargados: (1000, 21)
   Features: 20
   Target distribution: {1: 700, 2: 300}

📊 Información del dataset:
   Shape: (1000, 21)
   Memory usage: 0.81 MB
   Missing values: 0
   Duplicates: 0

📖 Primeras 5 filas:


,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,class
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2
